In [2]:
import earthaccess
import h5py
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib
# Set the backend to 'Agg' for non-interactive plotting, which is more reliable for scripts.
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import ssl
import os
import hvplot.xarray

In [3]:
# Log in to NASA Earthdata
auth = earthaccess.login(strategy="netrc", persist=True)
print("Logged in:", auth.authenticated)

Logged in: True


In [4]:
# --- Search and filter for PACE aerosol data spatiotemporally ---
results = earthaccess.search_data(
    short_name="PACE_SPEXONE_L2_AER_RTAPLAND",
    version="3.0",
    # cloud_hosted=True,
    bounding_box=(-23, 3, 24, 44), #WSEN
    temporal=("2025-04-01", "2025-04-01"),
    count=100  # Limit to files for simplicity
    # clouds = (0, 50) cloud cover if needs but for OCI
)
# --- number of filtered PACE aerosol files ---
print("Found", len(results), "files:")

Found 5 files:


In [5]:
paths = earthaccess.open(results)
paths

QUEUEING TASKS | :   0%|          | 0/5 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/5 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/5 [00:00<?, ?it/s]

[<File-like object S3FileSystem, ob-cumulus-prod-public/PACE_SPEXONE.20250401T104354.L2.RTAP_LD.V3_0.nc>,
 <File-like object S3FileSystem, ob-cumulus-prod-public/PACE_SPEXONE.20250401T121216.L2.RTAP_LD.V3_0.nc>,
 <File-like object S3FileSystem, ob-cumulus-prod-public/PACE_SPEXONE.20250401T121716.L2.RTAP_LD.V3_0.nc>,
 <File-like object S3FileSystem, ob-cumulus-prod-public/PACE_SPEXONE.20250401T122217.L2.RTAP_LD.V3_0.nc>,
 <File-like object S3FileSystem, ob-cumulus-prod-public/PACE_SPEXONE.20250401T135538.L2.RTAP_LD.V3_0.nc>]

In [6]:
ds_geophysical = xr.open_dataset(paths[0], group='geophysical_data')
ds_geophysical

<xarray.Dataset> Size: 44MB
Dimensions:                          (number_of_lines: 396,
                                      pixels_per_line: 29, wavelength: 34,
                                      wavelength3d: 18)
Dimensions without coordinates: number_of_lines, pixels_per_line, wavelength,
                                wavelength3d
Data variables: (12/123)
    wind_speed                       (number_of_lines, pixels_per_line) float32 46kB ...
    wind_speed_uncertainty           (number_of_lines, pixels_per_line) float32 46kB ...
    chla                             (number_of_lines, pixels_per_line) float32 46kB ...
    chla_uncertainty                 (number_of_lines, pixels_per_line) float32 46kB ...
    bpdf_scale                       (number_of_lines, pixels_per_line) float32 46kB ...
    bpdf_scale_uncertainty           (number_of_lines, pixels_per_line) float32 46kB ...
    ...                               ...
    alh_mode1                        (number_of_lines, pixels_per_line) float32 46kB ...
    alh_mode1_uncertainty            (number_of_lines, pixels_per_line) float32 46kB ...
    alh_mode2                        (number_of_lines, pixels_per_line) float32 46kB ...
    alh_mode2_uncertainty            (number_of_lines, pixels_per_line) float32 46kB ...
    alh_mode3                        (number_of_lines, pixels_per_line) float32 46kB ...
    alh_mode3_uncertainty            (number_of_lines, pixels_per_line) float32 46kB ...

In [16]:
ds_geolocation = xr.open_dataset(paths[0], group='geolocation_data')
ds_geolocation

<xarray.Dataset> Size: 1MB
Dimensions:                 (number_of_lines: 396, pixels_per_line: 29,
                             number_of_views: 5)
Dimensions without coordinates: number_of_lines, pixels_per_line,
                                number_of_views
Data variables:
    latitude                (number_of_lines, pixels_per_line) float32 46kB ...
    longitude               (number_of_lines, pixels_per_line) float32 46kB ...
    height                  (number_of_lines, pixels_per_line) float32 46kB ...
    height_std              (number_of_lines, pixels_per_line) float32 46kB ...
    fracday                 (number_of_lines, pixels_per_line) float32 46kB ...
    utc_date                (number_of_lines, pixels_per_line) float64 92kB ...
    pixel_id                (number_of_lines, pixels_per_line) float64 92kB ...
    sensor_zenith_angle     (number_of_lines, pixels_per_line, number_of_views) float32 230kB ...
    relative_azimuth_angle  (number_of_lines, pixels_per_line, number_of_views) float32 230kB ...
    scattering_angle        (number_of_lines, pixels_per_line, number_of_views) float32 230kB ...
    solar_zenith_angle      (number_of_lines, pixels_per_line) float32 46kB ...

In [ ]:
bpdf = ds['bpdf_scale']  # keep as xarray.DataArray
bpdf.hvplot.image()